# Проект e-learning: вариант 2

## Введение

Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (10 баллов)

3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (7 баллов) 

4. Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3). (8 баллов)

5. Используя pandas, в период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.  (15 баллов) 

6. Часто для качественного анализа аудитории используют подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Описание подхода можно найти тут. (23 балла)

Для решения задачи проведи предварительное исследование данных и сформулируй, что должно считаться курсом. Обосновать свой выбор ты можешь с помощью фактов сдачи экзаменов, распределения студентов и уникального идентификатора курса.

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

<div class="alert alert-block alert-success">
<b>
Устанавливаем необходимые пакеты
</b>
</div>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sbs

%matplotlib inline

<div class="alert alert-block alert-success">
<b>
Загрузим данные файлов из <em>Яндекс.Диск</em>. Дополнительно проверим данные на пропущенные значения и правильное определение типа данных.
</b>
</div>

In [2]:
from urllib.parse import urlencode
import requests

def read_yandexd_csv(url_groups, separator):
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    
    final_url_groups = base_url + urlencode(dict(public_key=url_groups))  
    
    response_groups = requests.get(final_url_groups)  
    
    download_url_groups = response_groups.json()['href']
    
    df = pd.read_csv(download_url_groups, sep=separator)
    
    return df

In [3]:
assessments = read_yandexd_csv('https://disk.yandex.ru/d/iSVP1sZ7AdW7TQ', ',')

In [4]:
assessments.shape

(206, 6)

In [5]:
assessments.isna().sum()

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64

In [6]:
assessments.dtypes

code_module           object
code_presentation     object
id_assessment          int64
assessment_type       object
date                 float64
weight               float64
dtype: object

In [7]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [8]:
assessments.query("code_module == 'AAA'")

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
5,AAA,2013J,1757,Exam,NaN,100.0
6,AAA,2014J,1758,TMA,19.0,10.0
7,AAA,2014J,1759,TMA,54.0,20.0
8,AAA,2014J,1760,TMA,117.0,20.0
9,AAA,2014J,1761,TMA,166.0,20.0


<div class="alert alert-class alert-info">
<b>

**assessments.csv** — этот файл содержит информацию об оценках в тесте. Обычно каждый предмет в семестре включает ряд тестов с оценками, за которыми следует заключительный экзаменационный тест (экзамен).
-   **code_module** — идентификационный код предмета.
-   **code_presentation** — семестр (Идентификационный код).
-   **id_assessment** — тест (Идентификационный номер ассессмента).
-   **assessment_type** — тип теста. Существуют три типа оценивания: оценка преподавателя (TMA), компьютерная оценка (СМА), экзамен по курсу (Exam).
-   **date** — информация об окончательной дате сдачи теста. Рассчитывается как количество дней с момента начала семестра. Дата начала семестра имеет номер 0 (ноль).
-   **weight** — вес теста в % в оценке за курс. Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%.

</b>
</div>

In [9]:
courses = read_yandexd_csv('https://disk.yandex.ru/d/jmJxR8yhwCXlaw', ',')

In [10]:
courses.shape

(22, 3)

In [11]:
courses.isna().sum()

code_module                   0
code_presentation             0
module_presentation_length    0
dtype: int64

In [12]:
courses.dtypes

code_module                   object
code_presentation             object
module_presentation_length     int64
dtype: object

In [13]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


<div class="alert alert-class alert-info">
<b>

**courses.csv** — файл содержит список предметов по семестрам.
-   **code_module** — предмет (идентификационный код).
-   **code_presentation** — семестр (идентификационный код).
-   **module_presentation_length** — продолжительность семестра в днях.

</b>
</div>

In [14]:
studentAssessment = read_yandexd_csv('https://disk.yandex.ru/d/DYd4M5b_8RETsA', ',')

In [15]:
studentAssessment.shape

(173912, 5)

In [16]:
studentAssessment.isna().sum()

id_assessment       0
id_student          0
date_submitted      0
is_banked           0
score             173
dtype: int64

In [17]:
studentAssessment.dtypes

id_assessment       int64
id_student          int64
date_submitted      int64
is_banked           int64
score             float64
dtype: object

In [18]:
studentAssessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


<div class="alert alert-class alert-info">
<b>

**studentAssessment.csv** — этот файл содержит результаты тестов студентов. Если учащийся не отправляет работу на оценку, результат не записывается в таблицу.
-   **id_assessment** — тест (идентификационный номер).
-   **id_student** — идентификационный номер студента.
-   **date_submitted** — дата сдачи теста студентом, измеряемая как количество дней с начала семестра.
-   **is_banked** — факт перезачета теста с прошлого семестра (иногда курсы перезачитывают студентам, вернувшимся из академического отпуска).
-   **score** — оценка учащегося в этом тесте. Диапазон составляет от 0 до 100. Оценка ниже 40 неудачная/неуспешная сдача теста.

</b>
</div>

In [19]:
studentRegistration = read_yandexd_csv('https://disk.yandex.ru/d/lqHnH8NOZ458iw', ',')

In [20]:
studentRegistration.shape

(32593, 5)

In [21]:
studentRegistration.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

In [22]:
studentRegistration.dtypes

code_module             object
code_presentation       object
id_student               int64
date_registration      float64
date_unregistration    float64
dtype: object

In [23]:
studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


<div class="alert alert-class alert-info">
<b>

**studentRegistration.csv** — этот файл содержит информацию о времени, когда студент зарегистрировался для прохождения курса в семестре.
-   **code_module** — предмет (идентификационный код).
-   **code_presentation** — семестр (идентификационный код)
-   **id_student** — идентификационный номер студента.
-   **date_registration** — дата регистрации студента. Это количество дней, измеренное от начала семестра (например, отрицательное значение -30 означает, что студент зарегистрировался на прохождение курса за 30 дней до его начала).
-   **date_unregistration** — дата отмены регистрации студента с предмета. У студентов, окончивших курс, это поле остается пустым.

</b>
</div>

In [24]:
#№ строки, где есть значения NaN в столбце score - экзамен еще не сдан
studentAssessment[studentAssessment.isna().any(axis=1)]

,id_assessment,id_student,date_submitted,is_banked,score
215,1752,721259,22,0,NaN
937,1754,260355,127,0,NaN
2364,1760,2606802,180,0,NaN
3358,14984,186780,77,0,NaN
3914,14984,531205,26,0,NaN
...,...,...,...,...,...
148929,34903,582670,241,0,NaN
159251,37415,610738,87,0,NaN
166390,37427,631786,221,0,NaN
169725,37435,648110,62,0,NaN


<div class="alert alert-block alert-info">
<b>
Прежде, чем приступить к анализу данных, необходимо определить что такое курс в рамках этих данных.
</b>
</div>

In [25]:
courses

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240
5,BBB,2014B,234
6,CCC,2014J,269
7,CCC,2014B,241
8,DDD,2013J,261
9,DDD,2014J,262


In [26]:
# Проверим, что номера тестов уникальны для каждого семестра (каждому семестру и курсу присвоен уникальный номер тестов)
assessments['id_assessment'].nunique()

206

In [27]:
assessments.shape

(206, 6)

In [28]:
assessments.query("code_presentation == '2013J'")

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
5,AAA,2013J,1757,Exam,NaN,100.0
24,BBB,2013J,15003,CMA,54.0,1.0
25,BBB,2013J,15004,CMA,96.0,1.0
26,BBB,2013J,15005,CMA,131.0,1.0
27,BBB,2013J,15006,CMA,166.0,1.0


In [29]:
# количество студентов зарегистрированных на курс и не отменивших регистрацию
studentRegistration[studentRegistration['date_unregistration'].isna()].\
                    groupby(['code_module', 'code_presentation']).\
                    agg({'id_student':'count'}).\
                    rename(columns={'id_student':'count_student'}).\
                    reset_index()

,code_module,code_presentation,count_student
0,AAA,2013J,323
1,AAA,2014J,299
2,BBB,2013B,1262
3,BBB,2013J,1590
4,BBB,2014B,1124
5,BBB,2014J,1556
6,CCC,2014B,1038
7,CCC,2014J,1449
8,DDD,2013B,872
9,DDD,2013J,1254


In [30]:
studentRegistration[studentRegistration['date_unregistration'].isna()].shape[0]

22521

In [31]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [32]:
# Проверим, совпадает ли длительность семестра для разных годов
courses.groupby(['code_module', 'code_presentation']).\
        agg({'module_presentation_length':'sum'})

module_presentation_length
code_module code_presentation                            
AAA         2013J                                     268
            2014J                                     269
BBB         2013B                                     240
            2013J                                     268
            2014B                                     234
            2014J                                     262
CCC         2014B                                     241
            2014J                                     269
DDD         2013B                                     240
            2013J                                     261
            2014B                                     241
            2014J                                     262
EEE         2013J                                     268
            2014B                                     241
            2014J                                     269
FFF         2013B                                     240
            2013J                                     268
            2014B                                     241
            2014J                                     269
GGG         2013J                                     261
            2014B                                     241
            2014J                                     269

Вес теста за курс: экзамен - 100%, сумма всех остальных оценок тоже 100%


In [33]:
assessments.query("assessment_type != 'Exam' and code_module == 'AAA' and code_presentation == '2013J'").groupby(['code_module', 'code_presentation']).agg({'weight':'sum'})

,,weight
code_module,code_presentation,
AAA,2013J,100.0


In [34]:
assessments.query("assessment_type != 'Exam' and code_module == 'BBB' and code_presentation == '2013J'").groupby(['code_module', 'code_presentation']).agg({'weight':'sum'})

,,weight
code_module,code_presentation,
BBB,2013J,100.0


In [35]:
# По предмету GGG имел вес только экзамен

assessments.query('code_module == "GGG"')

,code_module,code_presentation,id_assessment,assessment_type,date,weight
176,GGG,2013J,37418,CMA,229.0,0.0
177,GGG,2013J,37419,CMA,229.0,0.0
178,GGG,2013J,37420,CMA,229.0,0.0
179,GGG,2013J,37421,CMA,229.0,0.0
180,GGG,2013J,37422,CMA,229.0,0.0
181,GGG,2013J,37423,CMA,229.0,0.0
182,GGG,2013J,37415,TMA,61.0,0.0
183,GGG,2013J,37416,TMA,124.0,0.0
184,GGG,2013J,37417,TMA,173.0,0.0
185,GGG,2013J,37424,Exam,229.0,100.0


In [36]:
studentRegistration['id_student'].value_counts().reset_index().groupby('count').agg({'count'})

,id_student
,count
count,
1,25247
2,3293
3,221
4,23
5,1


Судя по подсчету id_student, значения не уникальные и могут повторяться. Возможно, это студенты, которые были переведены на другой курс

In [37]:
studentRegistration.query('id_student == 562064')

,code_module,code_presentation,id_student,date_registration,date_unregistration
15292,DDD,2013J,562064,-198.0,NaN
26658,FFF,2014B,562064,-255.0,135.0
28199,FFF,2014J,562064,-22.0,NaN


Изучив данные по таблице assessments, увидел, что в каждом семестре есть похожие предметы 
и все они заканчиваются экзаменом в конце.
Я сделал таблицу по количеству студентов в каждом семестре по каждому предмету.
По ней я не увидел прямой закономерности количества студентов. Обычно, если курс начался и был набор студентов на него, дальше они отсеиваются, если не сдают экзамен.
Здесь же, такой закономерности нет.

Но дальше я обратил внимание на столбец weight и информацию по нему: "вес теста в % в оценке за курс. 
Обычно экзамены рассматриваются отдельно и имеют вес 100%; сумма всех остальных оценок составляет 100%."
Т.к. за вес экзамена по умолчанию 100%, мне оставалось проверить, будет ли сумма тестов за предмет в каждом семестре равна 100%. Сделал проверку для 4х предметов в разных семестрах и оказалось, что их вес 100%

Из этого я делаю вывод, что 1 предмет в отдельном семестре - это 1 курс.
Теперь можно ответить на 1 вопрос

<div class="alert alert-block alert-info"
<b>
    
1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене).

</b>
</div>

In [38]:
# Доказательство, что в данных нет оценок за экзамены
assessments.query("assessment_type == 'Exam'")

,code_module,code_presentation,id_assessment,assessment_type,date,weight
5,AAA,2013J,1757,Exam,NaN,100.0
11,AAA,2014J,1763,Exam,NaN,100.0
23,BBB,2013B,14990,Exam,NaN,100.0
35,BBB,2013J,15002,Exam,NaN,100.0
47,BBB,2014B,15014,Exam,NaN,100.0
53,BBB,2014J,15025,Exam,NaN,100.0
62,CCC,2014B,24290,Exam,NaN,100.0
63,CCC,2014B,40087,Exam,NaN,100.0
72,CCC,2014J,24299,Exam,NaN,100.0
73,CCC,2014J,40088,Exam,NaN,100.0


In [39]:
# Взял 3 выборочных значения и не нашла совпадений для всех них
studentAssessment.query("id_assessment == 1757")

,id_assessment,id_student,date_submitted,is_banked,score


In [40]:
studentAssessment.query("id_assessment == 15014")

,id_assessment,id_student,date_submitted,is_banked,score


In [41]:
studentAssessment.query("id_assessment == 34885")

,id_assessment,id_student,date_submitted,is_banked,score


Поэтому я предположил, что зачет перед сдачей экзамена - это наличие сданных тестов по этому предмету на проходной балл(40 и более баллов)

In [42]:
# Какие тесты сдавал отдельно взятый студент с id 6516
studentAssessment.query("id_student == 6516")

,id_assessment,id_student,date_submitted,is_banked,score
1756,1758,6516,17,0,60.0
1970,1759,6516,51,0,48.0
2376,1760,6516,116,0,63.0
2685,1761,6516,164,0,61.0
2871,1762,6516,210,0,77.0


In [43]:
studentAssessment.query("score == 'NaN'")

,id_assessment,id_student,date_submitted,is_banked,score


In [44]:
# ! Учесть тех, кто не сдал какой то экзамен (173 строки!)
# замена на ноль
studentAssessment['score'] = studentAssessment['score'].fillna(0)

In [45]:
# Проверка, что замена на 0 прошла успешно
studentAssessment.query("score == 'NaN'")

,id_assessment,id_student,date_submitted,is_banked,score


In [46]:
# Группируем по id студента, потому что он уникален для каждого студента на любом курсе (доказано выше)
# И получаем все оценки по тестам, которые студент сдавал по своему предмету (курсу) 
student_scores = studentAssessment.groupby('id_student').agg({'score': lambda x: x.tolist()}).reset_index()
student_scores

,id_student,score
0,6516,"[60.0, 48.0, 63.0, 61.0, 77.0]"
1,8462,"[93.0, 83.0, 87.0, 93.0, 83.0, 83.0, 87.0]"
2,11391,"[78.0, 85.0, 80.0, 85.0, 82.0]"
3,23629,"[67.0, 63.0, 100.0, 100.0]"
4,23698,"[94.0, 60.0, 69.0, 67.0, 78.0, 93.0, 73.0, 56...."
...,...,...
23364,2698251,"[69.0, 76.0, 53.0, 67.0, 62.0, 36.0, 44.0]"
23365,2698257,"[58.0, 64.0, 76.0, 67.0, 74.0]"
23366,2698535,"[23.0, 20.0, 41.0, 13.0, 61.0, 58.0, 62.0, 36.0]"
23367,2698577,"[100.0, 54.0, 56.0, 58.0, 54.0]"


Группируем по id студента, потому что он уникален для каждого студента на людом курсе (доказано выше)
И оставляем минимальное значение среди оценок за тесты

In [47]:
student_scores_min = studentAssessment.groupby('id_student').agg({'score': 'min'}).rename(columns={'score': 'min_score'}).reset_index()
student_scores_min

,id_student,min_score
0,6516,48.0
1,8462,83.0
2,11391,78.0
3,23629,63.0
4,23698,56.0
...,...,...
23364,2698251,36.0
23365,2698257,58.0
23366,2698535,13.0
23367,2698577,54.0


In [48]:
# Количество студентов, которые успешно сдали 1 курс. (Т.е. сдали тесты по своему предмету > чем на 39 баллов и были допущены к экзамену)
student_scores_min.query("min_score >= 40").shape[0]

18402

<div class="alert alert-block alert-info"
<b>
    
2. Выявить самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью (завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен).
Под экзаменом здесь я понимаю отдельный тест

</b>
</div>

In [49]:
# количество попыток сдать каждый экзамен
count_pass = studentAssessment.groupby(['id_assessment']).agg({'score': 'count'}).reset_index()
count_pass = count_pass.rename(columns={'score': 'count_try_pass'})
count_pass.head()

,id_assessment,count_try_pass
0,1752,359
1,1753,342
2,1754,331
3,1755,303
4,1756,298


In [50]:
# количество успешных попыток сдать каждый экзамен
count_pozitiv_pass = studentAssessment.query("score >= 40").groupby(['id_assessment']).agg({'score': 'count'}).reset_index()
count_pozitiv_pass = count_pozitiv_pass.rename(columns={'score': 'count_pozitiv_pass'})
count_pozitiv_pass.head()

,id_assessment,count_pozitiv_pass
0,1752,352
1,1753,330
2,1754,325
3,1755,294
4,1756,290


In [51]:
# Объединяю 2 таблицы, чтобы собрать вместе количество попыток и количество успешных попыток сдать экзамен
count_pass = count_pass.merge(count_pozitiv_pass, how = 'inner', on = 'id_assessment')
count_pass.head()

,id_assessment,count_try_pass,count_pozitiv_pass
0,1752,359,352
1,1753,342,330
2,1754,331,325
3,1755,303,294
4,1756,298,290


In [52]:
# Создаю столбец, в котором считаю % успешности сдачи каждого теста
count_pass['complete'] = count_pass.count_pozitiv_pass / count_pass.count_try_pass * 100
count_pass.head()

,id_assessment,count_try_pass,count_pozitiv_pass,complete
0,1752,359,352,98.050139
1,1753,342,330,96.491228
2,1754,331,325,98.187311
3,1755,303,294,97.029703
4,1756,298,290,97.315436


In [53]:
# Добавляю в имеющуюся таблицу данные из таблицы assessments, чтобы у теста был предмет, к которому он относится
count_pass_module = count_pass.merge(assessments, on = "id_assessment")
count_pass_module.head()

,id_assessment,count_try_pass,count_pozitiv_pass,complete,code_module,code_presentation,assessment_type,date,weight
0,1752,359,352,98.050139,AAA,2013J,TMA,19.0,10.0
1,1753,342,330,96.491228,AAA,2013J,TMA,54.0,20.0
2,1754,331,325,98.187311,AAA,2013J,TMA,117.0,20.0
3,1755,303,294,97.029703,AAA,2013J,TMA,166.0,20.0
4,1756,298,290,97.315436,AAA,2013J,TMA,215.0,30.0


In [54]:
# Поиск самого простого экзамена - который сдают легче всего - в КАЖДОМ курсе
# Если таких тестов несколько, они все вносятся в таблицу
result_max = count_pass_module.groupby(['code_module','code_presentation']).apply(lambda x: x[x['complete'] == x['complete'].max()])
result_max

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_13240\499876161.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_max = count_pass_module.groupby(['code_module','code_presentation']).apply(lambda x: x[x['complete'] == x['complete'].max()])


id_assessment  count_try_pass  \
code_module code_presentation                                      
AAA         2013J             2             1754             331   
            2014J             5             1758             337   
BBB         2013B             16           14991            1189   
            2013J             27           15003            1490   
            2014B             38           15015            1054   
            2014J             44           15021            1586   
CCC         2014B             53           24287            1029   
            2014J             57           24291            1428   
DDD         2013B             74           25342             906   
            2013J             81           25349            1314   
            2014B             87           25355             930   
            2014J             95           25363            1325   
EEE         2013J             102          30710             748   
            2014B             105          30714             533   
            2014J             110          30720             826   
FFF         2013B             118          34865            1091   
            2013J             131          34879            1352   
            2014B             143          34892             857   
                              147          34896             747   
            2014J             157          34907            1234   
GGG         2013J             165          37419             685   
            2014B             174          37429             559   
            2014J             183          37439             504   

                                   count_pozitiv_pass    complete code_module  \
code_module code_presentation                                                   
AAA         2013J             2                   325   98.187311         AAA   
            2014J             5                   333   98.813056         AAA   
BBB         2013B             16                 1186   99.747687         BBB   
            2013J             27                 1488   99.865772         BBB   
            2014B             38                 1051   99.715370         BBB   
            2014J             44                 1567   98.802018         BBB   
CCC         2014B             53                  957   93.002915         CCC   
            2014J             57                 1352   94.677871         CCC   
DDD         2013B             74                  882   97.350993         DDD   
            2013J             81                 1259   95.814307         DDD   
            2014B             87                  876   94.193548         DDD   
            2014J             95                 1301   98.188679         DDD   
EEE         2013J             102                 744   99.465241         EEE   
            2014B             105                 527   98.874296         EEE   
            2014J             110                 823   99.636804         EEE   
FFF         2013B             118                1089   99.816682         FFF   
            2013J             131                1352  100.000000         FFF   
            2014B             143                 857  100.000000         FFF   
                              147                 747  100.000000         FFF   
            2014J             157                1234  100.000000         FFF   
GGG         2013J             165                 684   99.854015         GGG   
            2014B             174                 559  100.000000         GGG   
            2014J             183                 503   99.801587         GGG   

                                  code_presentation assessment_type   date  \
code_module code_presentation                                                
AAA         2013J             2               2013J             TMA  117.0   
            2014J             5               2014J             TMA   19.0

In [55]:
# Поиск самого сложного экзамена - который сдают хуже всего - в КАЖДОМ курсе
# Если таких тестов несколько, они все вносятся в таблицу
result_min = count_pass_module.groupby(['code_module','code_presentation']).apply(lambda x: x[x['complete'] == x['complete'].min()])
result_min

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_13240\507545333.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_min = count_pass_module.groupby(['code_module','code_presentation']).apply(lambda x: x[x['complete'] == x['complete'].min()])


id_assessment  count_try_pass  \
code_module code_presentation                                      
AAA         2013J             1             1753             342   
            2014J             6             1759             317   
BBB         2013B             13           14987             931   
            2013J             23           14998            1316   
            2014B             37           15013             684   
            2014J             43           15020            1776   
CCC         2014B             52           24286            1346   
            2014J             65           24299            1168   
DDD         2013B             71           25339             503   
            2013J             85           25353             845   
            2014B             92           25360             450   
            2014J             98           25366            1043   
EEE         2013J             103          30711             682   
            2014B             108          30717             372   
            2014J             111          30721             770   
FFF         2013B             115          34862            1076   
            2013J             127          34875            1402   
            2014B             141          34890             715   
            2014J             153          34903            1137   
GGG         2013J             168          37422             576   
            2014B             177          37432             467   
            2014J             181          37437             500   

                                   count_pozitiv_pass   complete code_module  \
code_module code_presentation                                                  
AAA         2013J             1                   330  96.491228         AAA   
            2014J             6                   297  93.690852         AAA   
BBB         2013B             13                  887  95.273899         BBB   
            2013J             23                 1281  97.340426         BBB   
            2014B             37                  646  94.444444         BBB   
            2014J             43                 1040  58.558559         BBB   
CCC         2014B             52                 1089  80.906389         CCC   
            2014J             65                 1019  87.243151         CCC   
DDD         2013B             71                  392  77.932406         DDD   
            2013J             85                  664  78.579882         DDD   
            2014B             92                  334  74.222222         DDD   
            2014J             98                  880  84.372004         DDD   
EEE         2013J             103                 657  96.334311         EEE   
            2014B             108                 351  94.354839         EEE   
            2014J             111                 750  97.402597         EEE   
FFF         2013B             115                1011  93.959108         FFF   
            2013J             127                1295  92.368046         FFF   
            2014B             141                 668  93.426573         FFF   
            2014J             153                1065  93.667546         FFF   
GGG         2013J             168                 552  95.833333         GGG   
            2014B             177                 431  92.291221         GGG   
            2014J             181                 472  94.400000         GGG   

                                  code_presentation assessment_type   date  \
code_module code_presentation                                                
AAA         2013J             1               2013J             TMA   54.0   
            2014J             6               2014J             TMA   54.0   
BBB         2013B             13              2013B             TMA  124.0   
            2013J             23              2013J             TMA   96.0   
            2

In [56]:
# Объединение таблиц "studentAssessment" и "assessments" по общему ключу "id_assessment", чтобы получить данные по каждому 
# предмету и тестам в рамках него у каждого студента
all_scores = pd.merge(assessments, studentAssessment, on='id_assessment')
all_scores.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0
1,AAA,2013J,1752,TMA,19.0,10.0,28400,22,0,70.0
2,AAA,2013J,1752,TMA,19.0,10.0,31604,17,0,72.0
3,AAA,2013J,1752,TMA,19.0,10.0,32885,26,0,69.0
4,AAA,2013J,1752,TMA,19.0,10.0,38053,19,0,79.0


In [57]:
# Среднее количество дней сдачи тестов по каждому предмету
mean_date_subm = all_scores.groupby(['code_module']).agg({'date_submitted': 'mean'}).reset_index()
mean_date_subm = mean_date_subm.rename(columns={'date_submitted': 'avg_date_submitted'})
mean_date_subm

,code_module,avg_date_submitted
0,AAA,109.354081
1,BBB,104.702524
2,CCC,116.598522
3,DDD,106.647842
4,EEE,90.896491
5,FFF,123.823515
6,GGG,152.757671


<div class="alert alert-block alert-info"
<b>
    
4. Выявить самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3).

</b>
</div>

In [58]:
# Количество регистраций по каждому предмету
count_reg = studentRegistration.groupby(['code_module']).agg({'id_student': 'count'}).reset_index()
count_reg = count_reg.rename(columns={'id_student': 'count_reg'}).sort_values('count_reg', ascending = False)
# ТОП-3 популярных предмета по количеству регистраций
count_reg.head(3)
# Количество регистраций, без учета тех, кто отменил в последствие свою регистрацию.

,code_module,count_reg
1,BBB,7909
5,FFF,7762
3,DDD,6272


In [59]:
# Количество регистраций на каждый курс, которые были отменены по каждому предмету
count_unreg = studentRegistration.query("date_unregistration != 'NaN'").groupby(['code_module']).agg({'id_student': 'count'}).reset_index()
count_unreg = count_unreg.rename(columns={'id_student': 'count_unreg'}).sort_values('count_unreg', ascending = False)
# ТОП-3 предметов с самым большим оттоком регистраций
count_unreg.head(3)

,code_module,count_unreg
1,BBB,7909
5,FFF,7762
3,DDD,6272


<div class="alert alert-block alert-info"
<b>
    
5. Используя pandas, в период с начала 2013 по конец 2014 выявить семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.

</b>
</div>

In [60]:
# Проверка, какие семестры есть в таблице assessments
assessments.code_presentation.value_counts()

code_presentation
2014J    57
2014B    57
2013J    53
2013B    39
Name: count, dtype: int64

In [61]:
# Проверка, какие семестры есть в таблице courses
courses.code_presentation.value_counts()

code_presentation
2014J    7
2013J    6
2014B    6
2013B    3
Name: count, dtype: int64

In [62]:
# Используем таблицу count_pass_module, в которой у нас уже есть информация о завершаемости каждого теста и 
# данные по семестрам/предметам
count_pass_module.head()

,id_assessment,count_try_pass,count_pozitiv_pass,complete,code_module,code_presentation,assessment_type,date,weight
0,1752,359,352,98.050139,AAA,2013J,TMA,19.0,10.0
1,1753,342,330,96.491228,AAA,2013J,TMA,54.0,20.0
2,1754,331,325,98.187311,AAA,2013J,TMA,117.0,20.0
3,1755,303,294,97.029703,AAA,2013J,TMA,166.0,20.0
4,1756,298,290,97.315436,AAA,2013J,TMA,215.0,30.0


In [63]:
# Для всех курсов в семестре посчитать среднюю завершаемость
avg_pass_sem_group = count_pass_module.groupby(['code_presentation']).agg({'complete': 'mean'}).reset_index()
avg_pass_sem_group = avg_pass_sem_group.rename(columns={'complete': 'mean_complete'}).sort_values('mean_complete', ascending = True)
# Семестр с самой низкой средней завершаемостью курсов
avg_pass_sem_group.head(1)

,code_presentation,mean_complete
0,2013B,94.616836


Семестр с самыми долгими средними сроками сдачи курсов
Используем таблицу all_scores, чтобы взять дату сдачи и сгруппировать уже по семестрам

In [64]:
# Средняя длительность сдачи всех курсов каждого семестра
avg_date_sem_group = all_scores.groupby(['code_presentation']).agg({'date_submitted': 'mean'}).reset_index()
avg_date_sem_group = avg_date_sem_group.rename(columns={'date_submitted': 'mean_date'}).sort_values('mean_date', ascending = False)
# Семестр с самой низкой средней завершаемостью курсов
avg_date_sem_group.head(1)

,code_presentation,mean_date
1,2013J,119.857195


<div class="alert alert-block alert-info"
<b>
    
6. Построить адаптированные RFM-кластеры студентов, чтобы качественно оценить аудиторию. Для каждого RFM-сегмента построить границы метрик recency, frequency и monetary для интерпретации этих кластеров.

</b>
</div>

In [65]:
# Проверка наличия отрицательных дат
all_scores.sort_values('date_submitted').head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
33276,BBB,2014B,15008,TMA,12.0,5.0,559381,-11,0,63.0
76237,DDD,2013J,25348,TMA,25.0,10.0,2472145,-10,0,85.0
22282,BBB,2013J,14996,TMA,19.0,5.0,610700,-9,0,75.0
104658,FFF,2013B,34865,CMA,222.0,0.0,539759,-8,0,98.0
105967,FFF,2013B,34866,CMA,222.0,0.0,557840,-8,0,88.0


In [66]:
# Сколько значений  = 0 баллов
all_scores.query("score == 0").shape[0]

502

In [67]:
# R - среднее время сдачи одного экзамена каждым студентом
avg_date_idstud = all_scores.groupby(['id_student']).agg({'date_submitted': 'mean'})
avg_date_idstud = avg_date_idstud.rename(columns={'date_submitted': 'mean_date_subm'}).reset_index()
avg_date_idstud = avg_date_idstud.sort_values('mean_date_subm')
avg_date_idstud.head()

,id_student,mean_date_subm
4064,423613,-4.0
9515,565851,-4.0
18413,653952,-4.0
230,61352,-3.0
8635,555297,-2.0


F - завершаемость курсов студентом - я рассчитывал, как процент успешно пройденных им тестов от общего числа тестов в курсе. Например, если студент успешно сдал 8 из 10 тестов, то его завершаемость будет 80%.

In [68]:
complete_course = student_scores
complete_course.head()

,id_student,score
0,6516,"[60.0, 48.0, 63.0, 61.0, 77.0]"
1,8462,"[93.0, 83.0, 87.0, 93.0, 83.0, 83.0, 87.0]"
2,11391,"[78.0, 85.0, 80.0, 85.0, 82.0]"
3,23629,"[67.0, 63.0, 100.0, 100.0]"
4,23698,"[94.0, 60.0, 69.0, 67.0, 78.0, 93.0, 73.0, 56...."


In [69]:
# Количество оценок больше 39 баллов у каждого студента
def count_scores_above_40 (score):
    count = sum(score >= 40 for score in score)
    return count

# Применяем функцию к каждой строке датафрейма
complete_course['count_scores_above_40'] = complete_course['score'].apply(count_scores_above_40)

complete_course.head()

,id_student,score,count_scores_above_40
0,6516,"[60.0, 48.0, 63.0, 61.0, 77.0]",5
1,8462,"[93.0, 83.0, 87.0, 93.0, 83.0, 83.0, 87.0]",7
2,11391,"[78.0, 85.0, 80.0, 85.0, 82.0]",5
3,23629,"[67.0, 63.0, 100.0, 100.0]",4
4,23698,"[94.0, 60.0, 69.0, 67.0, 78.0, 93.0, 73.0, 56....",9


In [70]:
# Общее количество оценок у каждого студента
def count_scores (score):
    count = sum(score >= 0 for score in score)
    return count

# Применяем функцию к каждой строке датафрейма
complete_course['count_scores'] = complete_course['score'].apply(count_scores)

complete_course.head()

,id_student,score,count_scores_above_40,count_scores
0,6516,"[60.0, 48.0, 63.0, 61.0, 77.0]",5,5
1,8462,"[93.0, 83.0, 87.0, 93.0, 83.0, 83.0, 87.0]",7,7
2,11391,"[78.0, 85.0, 80.0, 85.0, 82.0]",5,5
3,23629,"[67.0, 63.0, 100.0, 100.0]",4,4
4,23698,"[94.0, 60.0, 69.0, 67.0, 78.0, 93.0, 73.0, 56....",9,9


In [71]:
complete_course['complete_course'] = (complete_course['count_scores_above_40'] / complete_course['count_scores'] * 100).round(2)
complete_course.head()

,id_student,score,count_scores_above_40,count_scores,complete_course
0,6516,"[60.0, 48.0, 63.0, 61.0, 77.0]",5,5,100.0
1,8462,"[93.0, 83.0, 87.0, 93.0, 83.0, 83.0, 87.0]",7,7,100.0
2,11391,"[78.0, 85.0, 80.0, 85.0, 82.0]",5,5,100.0
3,23629,"[67.0, 63.0, 100.0, 100.0]",4,4,100.0
4,23698,"[94.0, 60.0, 69.0, 67.0, 78.0, 93.0, 73.0, 56....",9,9,100.0


In [72]:
complete_course.sort_values('complete_course').value_counts('complete_course').head()

complete_course
100.00    18402
80.00       740
0.00        396
85.71       374
50.00       370
Name: count, dtype: int64

In [73]:
#  M - среднее количество баллов, получаемое за экзамен - mean по каждому экзамену - для каждого студента
avg_score_idass_group = all_scores.groupby(['id_student']).agg({'score': 'mean'})
avg_score_idass_group = avg_score_idass_group.rename(columns={'score': 'mean_score'}).reset_index()
avg_score_idass_group = avg_score_idass_group.sort_values('mean_score')
avg_score_idass_group.head()

,id_student,mean_score
5018,480914,0.0
14571,614345,0.0
8138,549713,0.0
18459,654422,0.0
18590,656122,0.0


In [74]:
# Столбец айди студента, R, F, M, и их значения
stud_rfm = pd.merge(avg_date_idstud, complete_course[['id_student', 'complete_course']], on = 'id_student')
stud_rfm = pd.merge(stud_rfm, avg_score_idass_group, on = 'id_student')
stud_rfm.head()

,id_student,mean_date_subm,complete_course,mean_score
0,423613,-4.0,100.0,100.0
1,565851,-4.0,100.0,80.0
2,653952,-4.0,100.0,100.0
3,61352,-3.0,100.0,100.0
4,555297,-2.0,0.0,0.0


1. R - Средний срок сдачи экзаменов:

    Максимальная длительность семестров была равна 269 дней, поэтому за день окончания семестра я взял это число.
   - 5 - студенты, сдавшие экзамены раньше начала семестра - меньше 0 дней
   - 4 - студенты, сдавшие экзамены до середины курса - от 0 до 135 дней
   - 3 - студенты, сдавшие экзамены в середине курса или чуть позже - от 136 по 204 день
   - 2 - студенты, сдавшие экзамены в пределах установленного срока - от 205 дня до 269 дня
   - 1 - студенты, сдавшие экзамены после установленного срока - более 269 дней

2. F - Завершаемость курса:
   - 5 - студенты полностью завершившие курс - 100%
   - 4 - студенты, завершившие курс почти полностью - 80-99% (не сдавшие 1 тест)
   - 3 - студенты, завершившие курс с задержкой, но все же успешно - 50-79% (половину и более тестов)
   - 2 - студенты, сдавшие менее половины тестов - 1 - 49%
   - 1 - студенты, не сдавшие ни одного теста - 0%

3. M - Средняя оценка за экзамены:
   - 5 - студенты с высокими оценками - 100
   - 4 - студенты с средними оценками - 76-99
   - 3 - студенты с низкими оценками - 40 - 75
   - 2 - студенты, получившие оценки ниже неудовлетворительного уровня - 1 - 39
   - 1 - студенты, не сдавшие экзамены или получившие нулевые баллы - 0

In [75]:
# Методы для присвоения рангов от 1 до 5

def r_score(x):
    if x < 0:
        return 5
    elif 0 >= x and x <= 135:
        return 4
    elif 136 >= x and x  <= 204:
         return 3
    elif 205 >= x and x  <= 269:
        return 2
    else:
        return 1

In [76]:
def f_score(x):
    if x == 100:
        return 5
    elif x < 100 and x > 79:
        return 4
    elif x < 80 and x > 49:
        return 3
    elif x < 50 and x > 0:
        return 2
    else:
        return 1

In [77]:
def m_score(x):
    if x == 100:
        return 5
    elif x < 100 and x > 75:
        return 4
    elif x < 76 and x > 39:
        return 3
    elif x < 40 and x > 0:
        return 2
    else:
        return 1

In [78]:
stud_rfm['R'] = stud_rfm['mean_date_subm'].apply(lambda x: r_score(x))
stud_rfm['F'] = stud_rfm['complete_course'].apply(lambda x: f_score(x))
stud_rfm['M'] = stud_rfm['mean_score'].apply(lambda x: m_score(x))

In [79]:
stud_rfm.head()

,id_student,mean_date_subm,complete_course,mean_score,R,F,M
0,423613,-4.0,100.0,100.0,5,5,5
1,565851,-4.0,100.0,80.0,5,5,4
2,653952,-4.0,100.0,100.0,5,5,5
3,61352,-3.0,100.0,100.0,5,5,5
4,555297,-2.0,0.0,0.0,5,1,1


In [80]:
stud_rfm['RFM Score'] = stud_rfm['R'].map(str) + stud_rfm['F'].map(str) + stud_rfm['M'].map(str)
stud_rfm.head()

,id_student,mean_date_subm,complete_course,mean_score,R,F,M,RFM Score
0,423613,-4.0,100.0,100.0,5,5,5,555
1,565851,-4.0,100.0,80.0,5,5,4,554
2,653952,-4.0,100.0,100.0,5,5,5,555
3,61352,-3.0,100.0,100.0,5,5,5,555
4,555297,-2.0,0.0,0.0,5,1,1,511
